In [ ]:
import numpy as np
import scipy.stats as stats
import holoviews as hv
import panel as pn

In [ ]:
hv.extension('bokeh')
pn.extension()

# Set parameters for the target distribution
dim = 5  # Number of dimensions

# sigmas = np.array([1.0, 0.5, 0.75, 0.5, 1.0])
sigmas = np.abs(np.random.randn(dim)/5 + 1)
decay_rate = 0.5
rho = np.abs(np.arange(dim)[:, None] - np.arange(dim)[None, :])
rho = decay_rate**rho
cov = np.diag(sigmas) @ rho @ np.diag(sigmas)
image = hv.Image(cov).opts(
    cmap='viridis', colorbar=True, 
    width=400, height=400,
    invert_yaxis=True,
    data_aspect=1,
    frame_width=400,
    frame_height=400,
    )
target = stats.multivariate_normal(mean=np.zeros(dim), cov=cov)
image

In [ ]:
# Metropolis-Hastings algorithm implementation
def proposal(current, step_size):
    """Generate proposal from current state using Gaussian distribution."""
    return current + np.random.normal(0, np.sqrt(step_size), size=current.shape)

# MCMC parameters
n_burn = 1000     # Burn-in period
n_samples = 10000 # Number of samples to collect
step_size = 0.55  # Proposal step size (tuned for ~30% acceptance rate)

# Burn-in phase
current = np.zeros(dim)
for _ in range(n_burn):
    proposed = proposal(current, step_size)
    log_alpha = target.logpdf(proposed) - target.logpdf(current)
    if np.log(np.random.rand()) < log_alpha:
        current = proposed

# Sampling phase
samples = np.zeros((n_samples, dim))
acceptances = 0
for i in range(n_samples):
    proposed = proposal(current, step_size)
    log_alpha = target.logpdf(proposed) - target.logpdf(current)
    if np.log(np.random.rand()) < log_alpha:
        current = proposed
        acceptances += 1
    samples[i] = current

acceptance_rate = acceptances / n_samples
print(f"Acceptance rate: {acceptance_rate:.2f}")

# Prepare data for visualization
dim0_samples = samples[:, 0]  # First dimension samples for visualization


In [ ]:

# Create interactive visualization
def trace_plot(n):
    """Generate trace plot for first dimension up to n samples."""
    x = np.arange(n)
    y = dim0_samples[:n]
    return hv.Curve((x, y), 'Sample Index', 'Value').opts(
        title='Trace of First Dimension', width=500, height=300, color='#1f77b4')

def hist_plot(n):
    """Generate histogram for first dimension with true PDF overlay."""
    hist_vals, edges = np.histogram(dim0_samples[:n], bins=60, density=True)
    histogram = hv.Histogram((edges, hist_vals)).opts(
        alpha=0.5, title='Histogram of First Dimension', 
        width=500, height=300, color='#1f77b4')
    
    # True PDF curve
    x = np.linspace(-4, 4, 100)
    true_pdf = stats.norm.pdf(x)
    true_curve = hv.Curve((x, true_pdf), 'Value', 'Density').opts(
        color='red', line_width=2)
    
    return histogram * true_curve


In [ ]:
# Create slider for interactive control
n_slider = pn.widgets.IntSlider(
    name='Number of Samples', 
    start=1, 
    end=n_samples, 
    step=100, 
    value=1, 
    width=500
)

# Create dynamic dashboard
@pn.depends(n_slider.param.value)
def update_plots(n):
    return (trace_plot(n) + hist_plot(n)).cols(1)

# Assemble dashboard components
dashboard = pn.Column(
    pn.pane.Markdown(f"## Metropolis-Hastings Sampling Results"),
    pn.pane.Markdown(f"**Acceptance rate:** {acceptance_rate:.2f}"),
    n_slider,
    update_plots
)

dashboard.show()

In [ ]:
def lower_triangle_plot(n, num_dims=3):
    """Generate lower triangle plot with histograms on diagonal and scatter plots below."""
    plots = {}
    for i in range(num_dims):
        for j in range(i+1):
            if i == j:
                # Diagonal: Histogram with true PDF
                data = samples[:n, i]
                hist_vals, edges = np.histogram(data, bins=30, density=True)
                hist = hv.Histogram((edges, hist_vals)).opts(
                    xlabel=f'X{i}', ylabel='Density', 
                    width=200, height=200, toolbar=None)
                
                # Add true PDF
                x = np.linspace(-4, 4, 100)
                true_pdf = stats.norm.pdf(x)
                true_curve = hv.Curve((x, true_pdf)).opts(color='red', line_width=1)
                plots[(i, j)] = (hist * true_curve).opts(title=f'X{i} Distribution')
            else:
                # Lower triangle: Scatter plot with correlation
                x_data = samples[:n, j]
                y_data = samples[:n, i]
                true_corr = cov[i,j]  # Since variance=1, cov=correlation
                
                scatter = hv.Points((x_data, y_data)).opts(
                    xlabel=f'X{j}', ylabel=f'X{i}', alpha=0.05, size=2,
                    width=200, height=200, cmap='Blues', toolbar=None,
                    title=f'Corr: {true_corr:.2f}')
                
                # Add true correlation ellipse
                xx, yy = np.mgrid[-3:3:100j, -3:3:100j]
                pos = np.dstack((xx, yy))
                rv = stats.multivariate_normal([0,0], [[1,true_corr],[true_corr,1]])
                contours = hv.Contours((xx, yy, rv.pdf(pos))).opts(
                    cmap='reds', line_width=1)
                
                plots[(i, j)] = (scatter * contours).opts(show_legend=False)
    
    return hv.GridSpace(plots, kdims=['Row', 'Col']).opts(
        shared_xaxis=False, shared_yaxis=False,
        toolbar=None,
        title="Pairwise Relationships"
    )

# Update dashboard to include the new plot
@pn.depends(n_slider.param.value)
def update_plots(n):
    return pn.Column(
        pn.Row(trace_plot(n), hist_plot(n)),
        lower_triangle_plot(n, num_dims=dim)
    )

# Assemble updated dashboard
dashboard = pn.Column(
    pn.pane.Markdown("## Metropolis-Hastings Sampling Results"),
    pn.pane.Markdown(f"**Acceptance rate:** {acceptance_rate:.2f}"),
    n_slider,
    update_plots
)

In [ ]:
dashboard

In [ ]:

dashboard.show()